In [9]:
import pandas as pd
import config

In [2]:
pd.set_option('display.max_rows', 500)

In [3]:
df = pd.read_csv('processed/2023.csv')
df = df[~df['Empty Response']].replace("-", pd.NA)

In [39]:
df['(Upper) How satisfied are you with the education that Golden View Classical Academy provided this year?'].value_counts()

(Upper) How satisfied are you with the education that Golden View Classical Academy provided this year?
Extremely Satisfied    62
Satisfied              41
Somewhat Satisfied     14
Not Satisfied           3
Name: count, dtype: int64

In [27]:
df.head()

,Respondent ID,Collector ID,Start,End,IP Address,Email Address,First Name,Last Name,Custom Data,Submission Method,...,(Middle) How welcoming is the school community?,(Upper) How welcoming is the school community?,(Grammar) What makes GVCA a good choice for you and your family?,(Middle) What makes GVCA a good choice for you and your family?,(Upper) What makes GVCA a good choice for you and your family?,(Generic) What makes GVCA a good choice for you and your family?,(Grammar) Please provide us with examples of how GVCA can better serve you and your family.,(Middle) Please provide us with examples of how GVCA can better serve you and your family.,(Upper) Please provide us with examples of how GVCA can better serve you and your family.,(Generic) Please provide us with examples of how GVCA can better serve you and your family.
0,118223590845,449205862,01/17/2023 10:25:14 AM,02/14/2023 01:39:31 PM,NaN,NaN,NaN,NaN,NaN,Each parent or guardian will submit a separate...,...,Somewhat Welcoming,-,The reading curriculum. The lack of devices. ...,Middle school has been very challenging. The ...,-,Same,The class sizes seem to becoming more and more...,specific understanding of dyslexia would be ve...,-,Please put an end to swapping test and peer gr...
1,118234530973,449194285,02/01/2023 06:00:35 PM,02/01/2023 10:37:12 PM,NaN,NaN,NaN,NaN,NaN,All parents and guardians will coordinate resp...,...,-,Welcoming,Overall we have always been very pleased with ...,-,Golden View has made a lot of great changes in...,-,I am concerned with the changes in the math cu...,-,I love so many things that GVCA has to offer. ...,-
2,118234584005,449194285,02/01/2023 07:40:29 PM,02/01/2023 07:45:37 PM,NaN,NaN,NaN,NaN,NaN,All parents and guardians will coordinate resp...,...,-,Welcoming,-,-,We have been so blessed by GVCA’s constant eff...,-,-,-,It would be super helpful if there was a calen...,-
3,118234341372,449194285,02/01/2023 01:25:12 PM,02/01/2023 01:28:13 PM,NaN,NaN,NaN,NaN,NaN,All parents and guardians will coordinate resp...,...,-,-,We love our daughter's teacher and feel she ma...,-,-,We align fully with the school's emphasis on v...,-,-,-,It would be nice to have more parent mixers
4,118234309840,449194285,02/01/2023 12:54:18 PM,02/01/2023 01:04:04 PM,NaN,NaN,NaN,NaN,NaN,Each parent or guardian will submit a separate...,...,-,Welcoming,-,-,"Generally speaking, teachers seem to genuinely...",-,-,-,Be more transparent on what is expected on spe...,-


In [86]:
weight_by_parents = False
filters = {
    "Year 1 Families": pd.to_numeric(df["Years at GVCA"]) == 1,
    "Not Year 1 Families": pd.to_numeric(df["Years at GVCA"]) > 1,
    "Year 3 or Less Families": pd.to_numeric(df["Years at GVCA"]) <= 3,
    "Year 4 or More Families": pd.to_numeric(df["Years at GVCA"]) > 3,
    "Minority": df["Minority"] == "Yes",
    "Not Minority": df["Minority"] != "Yes",
    "Support": df["IEP, 504, ALP, or Read"] == "Yes",
    "Not Support": df["IEP, 504, ALP, or Read"] != "Yes",
}

def calculate_question_totals(df):
    results = []

    for question in config.questions_for_each_school_level:
        response_levels = config.question_responses.get(question, [])
        
        for response in response_levels:
            response_data = {"Question": question, "Response": response}
            
            # Calculate school-wide totals
            schoolwide_counts, schoolwide_total = _calculate_totals(df, question, response, config.levels, weight_by_parents)
            response_data.update(_format_counts_and_percentages("total", schoolwide_counts, schoolwide_total, response))
            
            # Calculate level-specific totals
            for level in config.levels:
                level_counts, level_total = _calculate_totals(df, question, response, [level], weight_by_parents)
                response_data.update(_format_counts_and_percentages(level, level_counts, level_total, response))
            
            # Calculate filtered totals
            for filter_name, filter_condition in filters.items():
                filtered_counts, filtered_total = _calculate_totals(df[filter_condition], question, response, config.levels, weight_by_parents)
                response_data.update(_format_counts_and_percentages(filter_name, filtered_counts, filtered_total, response))
            
            results.append(response_data)
    
    return pd.DataFrame(results)

def _calculate_totals(df, question, response, levels, weight_by_parents):
    """Helper to calculate counts and totals for given levels."""
    totals = {}
    overall_total = 0

    for level in levels:
        column_name = f"({level}) {question}"
        if column_name in df.columns:
            # Filter for rows matching the response
            filtered_df = df[df[column_name] == response]

            # Calculate level totals (weighted or unweighted)
            if weight_by_parents:
                response_sum = filtered_df["N Parents Represented"].astype(float).sum()
                level_total = df[~df[column_name].isna()]["N Parents Represented"].astype(float).sum()
            else:
                response_sum = len(filtered_df)
                level_total = len(df[column_name].dropna())

            # Add to totals
            totals[response] = totals.get(response, 0) + response_sum
            overall_total += level_total

    return totals, overall_total

def _format_counts_and_percentages(label, counts, total, response):
    """Helper to format counts and percentages for a given response."""
    count = counts.get(response, 0)
    percentage = (count / total) * 100 if total > 0 else 0
    return {f"N_{label}": count, f"%_{label}": percentage}

# Example usage
rolled_up_data = calculate_question_totals(df)
# rolled_up_data.to_csv("rolled_up_data.csv", index=False)
rolled_up_data

,Question,Response,N_total,%_total,N_Grammar,%_Grammar,N_Middle,%_Middle,N_Upper,%_Upper,...,N_Not Minority,%_Not Minority,N_Support,%_Support,N_Not Support,%_Not Support,N_Parents Respond Together,%_Parents Respond Together,N_Parents Respond Separately,%_Parents Respond Separately
0,How satisfied are you with the education that ...,Extremely Satisfied,310,62.626263,202,70.877193,46,51.111111,62,51.666667,...,278,62.612613,70,61.946903,240,62.827225,225,66.765579,85,53.797468
1,How satisfied are you with the education that ...,Satisfied,134,27.070707,58,20.350877,35,38.888889,41,34.166667,...,120,27.027027,28,24.778761,106,27.748691,89,26.409496,45,28.481013
2,How satisfied are you with the education that ...,Somewhat Satisfied,46,9.292929,23,8.070175,9,10.000000,14,11.666667,...,41,9.234234,14,12.389381,32,8.376963,22,6.528190,24,15.189873
3,How satisfied are you with the education that ...,Not Satisfied,5,1.010101,2,0.701754,0,0.000000,3,2.500000,...,5,1.126126,1,0.884956,4,1.047120,1,0.296736,4,2.531646
4,Given your children's education level at the b...,Extremely Satisfied,281,56.997972,184,65.017668,39,43.333333,58,48.333333,...,250,56.561086,62,55.855856,219,57.329843,203,60.597015,78,49.367089
5,Given your children's education level at the b...,Satisfied,159,32.251521,71,25.088339,38,42.222222,50,41.666667,...,146,33.031674,38,34.234234,121,31.675393,103,30.746269,56,35.443038
6,Given your children's education level at the b...,Somewhat Satisfied,47,9.533469,24,8.480565,13,14.444444,10,8.333333,...,40,9.049774,10,9.009009,37,9.685864,27,8.059701,20,12.658228
7,Given your children's education level at the b...,Not Satisfied,6,1.217039,4,1.413428,0,0.000000,2,1.666667,...,6,1.357466,1,0.900901,5,1.308901,2,0.597015,4,2.531646
8,"GVCA emphasizes 7 core virtues: Courage, Moder...",Strongly Reflected,215,43.259557,145,50.699301,26,28.571429,44,36.666667,...,189,42.471910,52,46.017699,163,42.447917,160,47.337278,55,34.591195
9,"GVCA emphasizes 7 core virtues: Courage, Moder...",Reflected,184,37.022133,110,38.461538,36,39.560440,38,31.666667,...,171,38.426966,32,28.318584,152,39.583333,128,37.869822,56,35.220126


(Grammar) How satisfied are you with the education that Golden View Classical Academy provided this year?
Extremely Satisfied    35
Somewhat Satisfied      1
Name: count, dtype: int64